# 곱셈 계층 구현
* `forward`(순전파) : `x * y`
* `backward`(역전파) : `dx = 미분값 * y`, `dy = 미분값 * x`
* 비고 : forward시에 입력되었던 `x, y`를 저장하고 있어야 한다. 그래야 Backward 할 때 반대방향으로 곱해줄 수 있다.

In [5]:
class MulLayer:

    # 레이어에서 사용할 변수를 생성자에서 준비
    def __init__(self):
        self.x = None
        self.y = None

    # 곱셈 레이어에서의 순전파 : x 와 y 를 곱하고 각 값을 저장
    def forward(self, x, y):
        self.x = x
        self.y = y

        # 순전파에서 진행되는 연산
        out = x * y
        return out

    # 곱셈 레이어의 역전파 : 들어온 미분 값을 x에는 y를 곱하고, y에는 x를 곱함
    def backward(self, dout):
        # dout : 다음 층에서 흘러 들어오는 미분값
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

In [6]:
# 곱셈 레이어 테스트
apple_per_cost = 100
apple_cnt = 2
tax = 1.1

mul_apple_layer = MulLayer()  # 사과의 총 가격을 구하는 레이어
mul_tax_layer = MulLayer()  # 소비세까지 적용시킨 가격을 구하는 레이어

In [8]:
# 순전파 수행
#  - 순서를 반드시 잘 지켜야 한다.
#  계획한 순서 그대로 레이어를 배치해서 입력값을 흘려보내야 한다.
#  순전파 수행 시에 A - B - C 순으로 계산을 했다면,
#  역전파 수행 시에는 C - B- A 순으로 미분값을 전달해야 한다.

apple_price = mul_apple_layer.forward(apple_per_cost, apple_cnt)
total_price = mul_tax_layer.forward(apple_price, tax)

print("사과의 최종 가격 : {:.0f}".format(total_price))

사과의 최종 가격 : 220


In [12]:
# 역전파
# 가장 마지막 값에 대한 미분값 생각하기
# d돈통 / d포스기 = 1

dprice = 1

dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple_per_cost, dapple_cnt = mul_apple_layer.backward(dapple_price)

print("사과 전체 가격에 대한 미분값 d돈통/d사과전체가격 : {}".format(dapple_price))
print("사과 1개 가격에 대한 미분값 d돈통/d사과1개가격 : {}".format(dapple_per_cost))
print("사과 개수에 대한 미분값 d돈통/d사과개수 : {}".format(dapple_cnt))
print("소비세에 대한 미분값 d돈통/d소비세 : {}".format(dtax))

사과 전체 가격에 대한 미분값 d돈통/d사과전체가격 : 1.1
사과 1개 가격에 대한 미분값 d돈통/d사과1개가격 : 2.2
사과 개수에 대한 미분값 d돈통/d사과개수 : 110.00000000000001
소비세에 대한 미분값 d돈통/d소비세 : 200


# 덧셈 레이어 구현
* `forward` : `x + y`
* `backward` : 뒷층에서 보낸 미분값에 *1만 하면 된다. `dx = dout * 1`, `dy = dout * 1`
  * `dx = dout * dforward/dx`
  * `dy = dout * dforward/dy`
* 비고 : 곱셈 계층과는 다르게 `forward`시에 입력된 값을 가지고 있지 않아도 된다. 역전파 시에는 미분값만 리턴하면 되니까

In [13]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

In [16]:
apple_per_cost = 100
apple_cnt = 2

orange_per_cost = 150
orange_cnt = 3

tax = 1.1

In [14]:
# 1 계층 : 사과/오렌지에 대한 국소적 계산

apple_layer = MulLayer()
orange_layer = MulLayer()

# 2 계층 : 과일의 전체 가격
fruit_layer = AddLayer()

# 3 계층 : 세금을 적용
tax_layer = MulLayer()

In [17]:
# 순전파

# 1 계층
apple_price = apple_layer.forward(apple_per_cost, apple_cnt)
orange_price = orange_layer.forward(orange_per_cost, orange_cnt)

# 2 계층
fruit_price = fruit_layer.forward(apple_price, orange_price)

# 3 계층
total_price = tax_layer.forward(fruit_price, tax)

In [19]:
total_price

715.0000000000001

In [20]:
# 역전파
d_total_price = 1

# 3 계층
d_fruit_price, d_tax = tax_layer.backward(d_total_price)

# 2 계층
d_apple_price, d_orange_price = fruit_layer.backward(d_fruit_price)

# 1 계층
d_apple_per_cost, d_apple_cnt = apple_layer.backward(d_apple_price)
d_orange_per_cost, d_orange_cnt = orange_layer.backward(d_orange_price)

In [21]:
print("소비세 미분 : {}".format(d_tax))

print("사과 전체 가격 미분 : {}".format(d_apple_price))
print("사과 개수 미분 : {}".format(d_apple_cnt))
print("사과 가격 미분 : {}".format(d_apple_per_cost))

print("오렌지 전체 가격 미분 : {}".format(d_orange_price))
print("오렌지 개수 미분 : {}".format(d_orange_cnt))
print("오렌지 가격 미분 : {}".format(d_orange_per_cost))

소비세 미분 : 650
사과 전체 가격 미분 : 1.1
사과 개수 미분 : 110.00000000000001
사과 가격 미분 : 2.2
오렌지 전체 가격 미분 : 1.1
오렌지 개수 미분 : 165.0
오렌지 가격 미분 : 3.3000000000000003
